In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import json
import pandas as pd
import numpy as np
import spacy
import inflect
from string import punctuation

inflect = inflect.engine()

data_path = "./../../data/"

In [4]:
nlp_lg = spacy.load('en_coref_lg')
nlp = nlp_lg

### SNLI

#### Load SNLI data, and use only entailment cases

In [5]:
snli_train_tsv = data_path +'snli/snli_1.0_train.txt'
snli_dev_tsv = data_path +'snli/snli_1.0_dev.txt'

snli_train_data = pd.read_csv(snli_train_tsv,delimiter='\t',encoding='utf-8')
snli_dev_data = pd.read_csv(snli_dev_tsv,delimiter='\t',encoding='utf-8')

snli_train_data = snli_train_data[['gold_label', 'sentence1', 'sentence2']]
snli_dev_data = snli_dev_data[['gold_label', 'sentence1', 'sentence2']]

In [6]:
snli_dev_data_entail = snli_dev_data.loc[snli_dev_data['gold_label'] == 'entailment']
snli_train_data_entail = snli_train_data.loc[snli_train_data['gold_label'] == 'entailment']

In [148]:
# This is a noisy, hand-labeled pronoun-dictionary
with open(data_path + "pronoun_dict", 'r') as f:
    pronoun_dict = json.load(f)

In [99]:
import sys
sys.path.append("./../")

from snli_parser import SNLI_PARSER

snli_parser = SNLI_PARSER(nlp_lg, pronoun_dict)

In [145]:
snli_dev_data_entail.shape

(3329, 3)

#### Generate a WNLI-style dataset

In [101]:
ENTAILMENT = 1
NOT_ENTAILMENT = 0

sentence1_list = []
sentence2_list = []
label_list = []
counter = 0

for ind, row in snli_dev_data_entail.iterrows():
    
    result = snli_parser.augment(row['sentence1'], row['sentence2'])
    if result is not None:
        (new_sent1, true_example, false_examples) = result
        
        # Save entailment results
        sentence1_list.append(new_sent1)
        sentence2_list.append(true_example)
        label_list.append(ENTAILMENT)
        
        # Save false entailment results
        sentence1_list.extend([new_sent1] * len(false_examples))
        sentence2_list.extend(false_examples)
        label_list.extend([NOT_ENTAILMENT] * len(false_examples))
    
    counter += 1
    if counter % 100 == 0:
        print(counter)
        
snli_dev_df = pd.DataFrame(list(zip(sentence1_list, sentence2_list, label_list)), columns=['sentence1','sentence2','label'])

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300


#### Get some examples of our dataset

In [149]:
for ind, row in snli_dev_df.iterrows():
    if random.random() < 0.01:
        print(row['sentence1'])
        print(row['sentence2'])
        print(row['label'])

A woman walks by a brick building that's covered with graffiti. She is outside.
A woman is outside.
1
A kid in a red and black coat is laying on his back in the snow with his arm in the air and a red sled is next to him. It is a cold day.
Coat is a cold day.
0
A brunette woman in a multicolored floral outfit and a pearl necklace is talking while holding a wineglass. She holds a glass
A woman holds a glass
1
A young man plays a racing video game. He is staring at a screen
A young man is staring at a screen
1
Man in leather jacket reading a book. He is reading
A man is reading
1
Several toddlers in a room with many toys, a yellow lamp, a window, gray carpeting and paintings. They are in a room with many toys, a yellow lamp, a window, gray carpeting and paintings.
Several toddlers are in a room with many toys, a yellow lamp, a window, gray carpeting and paintings.
1
The kid is wearing a blue jacket and standing in shallow beach water. He is at the beach.
The kid is at the beach.
1
A man t

### Try our WNLI scheme with coref model


In [112]:
import spacy
import sys
sys.path.append("./../")
from wnli import WNLI

In [131]:
ENTAILMENT = 1
NOT_ENTAILMENT = 0
MAJORITY = ENTAILMENT

In [134]:
print(snli_dev_df[(snli_dev_df['label'] == 1)].count())
print(snli_dev_df[(snli_dev_df['label'] == 0)].count())

sentence1    2507
sentence2    2507
label        2507
dtype: int64
sentence1    338
sentence2    338
label        338
dtype: int64


In [123]:
snli_dev_df[(snli_dev_df['sentence2'] == 'They got ready to cut the cake')]

row = snli_dev_df.iloc[1221]
print(row['sentence1'])
print(row['sentence2'])

People getting ready to cut a birthday cake. They got ready to cut the cake
They got ready to cut the cake


In [141]:
snli_dev_majority = WNLI(nlp_lg, snli_dev_df, majority=MAJORITY, use_coref=False, debug=False)
snli_dev_majority_labels = snli_dev_majority.predict()
snli_dev_majority_score = snli_dev_majority.score(snli_dev_majority_labels)

snli_dev = WNLI(nlp_lg, snli_dev_df, majority=MAJORITY, use_coref=True, debug=False)
snli_dev_labels = snli_dev.predict()
snli_dev_score = snli_dev.score(snli_dev_labels)

Could not use coref model for 789/2845 examples


In [144]:
print(snli_dev_majority_score)
print(snli_dev_score)

0.8811950790861159
0.7065026362038664


### MNLI

### RTE (Entailment)

### 